# 서울시 무인사진관 관련 시각화
1. 서울시 무인사진관 전체 분포 heatmap + dot 시각화
2. 서울시 무인사진관 밀집도 heatmap + dot 시각화
3. 서울시 무인사진관 밀집 heatmap + 대학교 marker 및 circle 시각화

In [5]:
# 필요 라이브러리 로드
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import requests
import json

## 사전 설정

In [6]:
# 사전 설정

# 서울 중심 좌표
center = [37.5642135, 127.0016985]

# 서울 구 경계 json: gu_geo
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
gu_geo = json.loads(c)

# 서울 동 경계 json: dong_geo
dong_geo = 'https://raw.githubusercontent.com/vuski/admdongkor/master/ver20221001/HangJeongDong_ver20221001.geojson'

# 서울시 무인사진관 전체 분포 heatmap + dot 시각화

In [2]:
# 서울시 무인사진관 위경도 포함 데이터 불러오기
df_1 = pd.read_csv('서울_무인사진관.csv')
df_1.head(2)

,Unnamed: 0,브랜드,매장,주소,링크,lat,lot,구,강남구
0,0,포토이즘,"포토이즘박스 익선동점사진,스튜디오",서울특별시 종로구 돈화문로11나길 32 1층,https://m.place.naver.com/place/1118162612/home,37.573568,126.990582,종로구,강남구 외
1,1,포토이즘,"포토이즘박스 건대점사진,스튜디오",서울특별시 광진구 아차산로29길 7 1층,https://m.place.naver.com/place/1855857891/home,37.542583,127.068929,광진구,강남구 외


In [3]:
# 브랜드별 컬러 적용
def color_select(brand):
    if brand == '인생네컷':
        return 'pink'
    elif brand == '포토이즘' :
        return 'black'
    elif brand == '포토그레이' :
        return 'green'
    else :
        return 'blue'

In [9]:
# 시각화
m = folium.Map(location=center, zoom_start=11)

# 배경 지도 API 설정
vworld_key = "1516082D-285B-3860-9DF6-96A8460B7AD1"

layer = "Base" # Base
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 히트맵
HeatMap(
    data=df_1[['lat', 'lot']], 
    radius=20,
).add_to(m)

# 브랜드별 무인사진관 dot 찍기
for _, row in df_1.iterrows():
        folium.Circle(location = [row['lat'], row['lot']],
               radius = 25,
               popup = row['브랜드'],
               color = color_select(row['브랜드'])
        ).add_to(m)
        
m

# 서울시 무인사진관 밀집도 heatmap + dot 시각화

In [10]:
df_2 = pd.read_csv('data/무인사진관_밀집도.csv')
df_2.head(2)

,브랜드,매장,주소,링크,lat,lot,구,강남구,위경도,주소_2,동,근방 매장수,근방 매장수 정규화
0,포토이즘,"포토이즘박스 익선동점사진,스튜디오",서울특별시 종로구 돈화문로11나길 32 1층,https://m.place.naver.com/place/1118162612/home,37.573568,126.990582,종로구,강남구 외,"37.5735683,126.9905822","4.5평 우동집, 돈화문로11나길, 익선동, 종로1·2·3·4가동, 종로구, 서울,...",익선동,4,0.75
1,포토이즘,"포토이즘박스 익선동점사진,스튜디오",서울특별시 종로구 돈화문로11나길 32 1층,https://m.place.naver.com/place/1118162612/home,37.573568,126.990582,종로구,강남구 외,"37.5735683,126.9905822","4.5평 우동집, 돈화문로11나길, 익선동, 종로1·2·3·4가동, 종로구, 서울,...",익선동,4,0.75


In [11]:
# 시각화
m = folium.Map(location=center, zoom_start=11)


# 지도 배경 vworld API key
vworld_key = "1516082D-285B-3860-9DF6-96A8460B7AD1"

# 지도 배경 설정
layer = "gray" # Base
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 무인사진관 밀집도 히트맵
HeatMap(
        data=df_2[['lat', 'lot']], 
        radius=20,
    ).add_to(m)

# 브랜드별 무인사진관 dot 찍기
for _, row in df_1.iterrows():
        folium.Circle(location = [row['lat'], row['lot']],
               radius = 25,
               popup = row['브랜드'],
               color = color_select(row['브랜드'])
        ).add_to(m)
        
m

# 서울시 무인사진관 dot + 대학교 marker 및 circle 시각화

In [13]:
# 대학
df_univ = pd.read_csv('data/학교 위치.csv')
df_univ.head(2)

,Unnamed: 0,학교종류,학교명,주소,행정구,행정동,lat,lot
0,0,일반대학,서울시립대학교,서울 동대문구 서울시립대로 163 (전농동 90번지),동대문구,휘경2동,37.585215,127.060126
1,1,전문대학(3년제),서울여자간호대학교,서울 서대문구 홍제3동 서울여자간호대학,서대문구,홍제2동,37.598824,126.948765


In [14]:
# 반경 표시용(고밀집/저밀집 구분) 무인사진관 위치 정보
df_pic = pd.read_csv('data/밀집기준최최종.csv')
df_pic.head(2)

,브랜드,매장,lat,lot,구,동,근방 매장수,근방 매장수 정규화,밀집여부,총_유동인구,남자10대_유동인구,남자20대_유동인구,남자30대_유동인구,남자40대이상_유동인구,여자10대_유동인구,여자20대_유동인구,여자30대_유동인구,여자40대이상_유동인구
0,포토이즘,"포토이즘박스 성수점사진,스튜디오",37.546677,127.042247,성동구,성수1가1동,3,0.5,저밀집,29447.526974,832.743109,2308.223173,3184.742959,7502.761074,1009.363032,2863.275506,3277.246129,7089.936747
1,하루필름,"하루필름 서울숲점셀프,대여스튜디오",37.546430,127.043585,성동구,성수1가1동,3,0.5,저밀집,29447.526974,832.743109,2308.223173,3184.742959,7502.761074,1009.363032,2863.275506,3277.246129,7089.936747


In [15]:
df_pic = df_pic.loc[df_pic['밀집여부']=='고밀집']

In [16]:
# 시각화

m = folium.Map(location=center, zoom_start=11, tiles=None) # 기본 지도 안 보이게

# 지도 배경 vworld API key
vworld_key = "1516082D-285B-3860-9DF6-96A8460B7AD1"

# 지도 배경 설정
layer = "Base" # Base
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True,
    opacity=0.5
).add_to(m)


# 대학교 반경 표시
for _, row in df_univ.iterrows():
    folium.Circle(location=[row['lat'], row['lot']],
                  color='orange',
                  fill_color='orange',
                  fill_opacity=0.1,
                 radius=500).add_to(m)

# 대학교 위치 마커
marker_cluster = MarkerCluster().add_to(m)

for lat, lot in zip(df_univ['lat'], df_univ['lot']):
    folium.Marker([lat, lot],
                 icon=folium.Icon(color="orange", icon='star')).add_to(marker_cluster)
    
# 브랜드별 컬러 적용
def color_select(brand):
    if brand == '인생네컷':
        return '#F08080'
    elif brand == '포토이즘' :
        return '#2F4F4F'
    elif brand == '포토그레이' :
        return '#228B22'
    else :
        return '#87CEFA'

# 무인사진관 위치 마커

marker_cluster = MarkerCluster().add_to(m)

for _, row in df_pic.iterrows():
    folium.Circle(location = [row['lat'], row['lot']],
           radius = 20,
            fill_color=color_select(row['브랜드']),
            fill_opacity=0.5,
           popup = row['브랜드'],
           color = color_select(row['브랜드'])
    ).add_to(m)
    
    
m